# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.0.1
['.ipynb_checkpoints', 'models', 'model_name', 'Student Notebook.ipynb', 'tmp', 'weights']


## Task 2: Create Model

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

## Task 4: Model Checkpoint During Training

In [4]:
checkpoint_dir = 'weights/'

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2, batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
            monitor='val_acc', save_weights_only=True, save_best_only=True
        )
    ]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 69us/sample - loss: 0.7006 - acc: 0.7602 - val_loss: 0.4995 - val_acc: 0.8230
Epoch 2/2
60000/60000 [==============================] - 2s 41us/sample - loss: 0.4294 - acc: 0.8493 - val_loss: 0.4255 - val_acc: 0.8476


In [5]:
os.listdir(checkpoint_dir)

['checkpoint',
 'epoch_01_acc_0.8230.data-00000-of-00001',
 'epoch_01_acc_0.8230.index',
 'epoch_02_acc_0.8476.data-00000-of-00001',
 'epoch_02_acc_0.8476.index']

## Task 5: Load Weights

In [6]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.470364187240601, 0.1192]


In [7]:
model.load_weights('weights/epoch_02_acc_0.8476')
print(model.evaluate(x_test, y_test, verbose=False))

[0.42554267122745515, 0.8476]


## Task 6: Saving Complete Model During Training

In [9]:
models_dir = 'models/'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2, batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 3s 50us/sample - loss: 0.6801 - acc: 0.7679 - val_loss: 0.4983 - val_acc: 0.8236
Epoch 2/2
60000/60000 [==============================] - 2s 37us/sample - loss: 0.4298 - acc: 0.8486 - val_loss: 0.4373 - val_acc: 0.8454


In [11]:
os.listdir(models_dir)

['.ipynb_checkpoints', 'epoch_01_acc_0.8236.h5', 'epoch_02_acc_0.8454.h5']

## Task 7: Load Models

In [12]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.4061312675476074, 0.0826]


In [15]:
model = tf.keras.models.load_model('models/epoch_02_acc_0.8454.h5')
print(model.evaluate(x_test, y_test, verbose=False))
model.summary()

[0.43726367220878604, 0.8454]
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 8: Manually Saving Weights and Models

In [16]:
model.save_weights('tmp/manuallyy_saved.w')
os.listdir('tmp')

['checkpoint',
 'manuallyy_saved.w.data-00000-of-00001',
 'manuallyy_saved.w.index']

In [17]:
model.save('tmp/manually_saved.h5')
os.listdir('tmp')

['checkpoint',
 'manuallyy_saved.w.data-00000-of-00001',
 'manuallyy_saved.w.index',
 'manually_saved.h5']

## Task 9: Exporting and Restoring SavedModel Format

In [19]:
model.save('model_name')
os.listdir('model_name/')

['assets', 'saved_model.pb', 'variables']

In [20]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4372636659145355, 0.8454]
